In [6]:
import itertools
import json
from pathlib import Path

import numpy as np
import pandas as pd 

from bellem.dvc.experiment import load_experiments

In [7]:
def product_experiment_configs(params_record):
    # Generate all possible combinations of parameters
    keys = params_record.keys()
    values = params_record.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [8]:
params_records = [
    {
        "params.qa.temperature": [0.0, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
        "params.qa.system_prompt": "standard/excellent-fewest.txt,standard/excellent-few.txt,standard/helpful-output-format-few.txt,standard/minimal-output-format-fewest.txt,standard/minimal-output-format-few-no-prio.txt,standard/minimal-output-format-few.txt,standard/minimal-output-format.txt".split(
            ","
        ),
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["empty.json", "standard-2-shot.json"],
        "params.run": [1, 2, 3],
    },
    # {
    #     "params.qa.temperature": [0.0, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
    #     "params.qa.system_prompt": "cot/format-minimal.txt,cot/format-reasoning.txt,cot/excellent-format-reasoning.txt,cot/format-think.txt".split(
    #         ","
    #     ),
    #     "params.qa.user_prompt_template": ["cq.txt"],
    #     "params.qa.few_shot_examples": ["empty.json"],
    #     "params.run": [1, 2, 3],
    # },
    # {
    #     "params.qa.temperature": [0.0, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
    #     "params.qa.system_prompt": "cte/excellent-format-fewest.txt,cte/excellent-format-few.txt,cte/format-few.txt,cte/format-relevant-few.txt".split(
    #         ","
    #     ),
    #     "params.qa.user_prompt_template": ["cq.txt"],
    #     "params.qa.few_shot_examples": "empty.json,cte-2-shot-delim-1.json,cte-2-shot-delim-2.json,cte-2-shot-delim-3.json".split(
    #         ","
    #     ),
    #     "params.run": [1, 2, 3],
    # },
]

exp_configs = [
    exp_config
    for params_record in params_records
    for exp_config in product_experiment_configs(params_record)
]
print(f"{len(exp_configs)} experiment configurations")

336 experiment configurations


In [9]:
target_params = exp_configs[0].keys()
target_params

dict_keys(['params.qa.temperature', 'params.qa.system_prompt', 'params.qa.user_prompt_template', 'params.qa.few_shot_examples', 'params.run'])

In [10]:
results_path = Path("experiments.jsonl")
if results_path.exists():
    with open(results_path) as f:
        experiments = [json.loads(line) for line in f]
else:
    experiments = []

print(f"{len(experiments)} experiments")
next(iter(experiments), None)

392 experiments


{'id': '3ddffbe7f93b656485c8735c86e935dc825f48c7',
 'name': 'worse-yolk',
 'params.dataset.path': 'bdsaglam/musique',
 'params.dataset.name': 'answerable',
 'params.dataset.split': 'validation',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.temperature': 2.0,
 'params.qa.system_prompt': 'cte/excellent-format-few',
 'params.qa.user_prompt_template': 'cq',
 'params.qa.few_shot_examples': 'empty.json',
 'params.run': 3,
 'metrics.exact_match': 0.0173769135,
 'metrics.f1': 0.0232922283,
 'metrics.2hops.exact_match': 0.0271565495,
 'metrics.2hops.f1': 0.0352128314,
 'metrics.3hops.exact_match': 0.0065789474,
 'metrics.3hops.f1': 0.0118271431,
 'metrics.4hops.exact_match': 0.0074074074,
 'metrics.4hops.f1': 0.0079561043,
 'params.qa.technique': 'CTE',
 'params.qa.n_shot': 0}

In [11]:
df = pd.DataFrame(experiments)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

392 experiments after preprocessing


In [12]:
existing_configs = df[target_params].to_dict(orient="records")
existing_configs[0]

{'params.qa.temperature': 2.0,
 'params.qa.system_prompt': 'cte/excellent-format-few',
 'params.qa.user_prompt_template': 'cq',
 'params.qa.few_shot_examples': 'empty.json',
 'params.run': 3}

In [13]:
run_counts = (
    df.groupby(["params.qa.system_prompt", "params.qa.user_prompt_template", "params.qa.few_shot_examples", "params.qa.temperature"])[
        "params.run"
    ]
    .aggregate(tuple)
    .reset_index()
)
mask = run_counts["params.run"].map(len) < 3
run_counts.loc[mask]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.temperature,params.run
72,standard/excellent-few,cq,standard-2-shot.json,0.0,"(3, 2)"


In [14]:
# find the missing configurations
missing_configs = [dict(kv) for kv in list({tuple(sorted(config.items())) for config in exp_configs} - {tuple(sorted(config.items())) for config in existing_configs})]
print(f"{len(missing_configs)} missing configurations")
missing_configs[0]


336 missing configurations


{'params.qa.few_shot_examples': 'standard-2-shot.json',
 'params.qa.system_prompt': 'standard/minimal-output-format-few-no-prio.txt',
 'params.qa.temperature': 0.5,
 'params.qa.user_prompt_template': 'cq.txt',
 'params.run': 2}

In [15]:
def make_command(exp_config):
    run = exp_config['params.run']
    temperature = exp_config['params.qa.temperature']
    system_prompt = exp_config['params.qa.system_prompt']
    user_prompt_template = exp_config['params.qa.user_prompt_template']
    few_shot_examples = exp_config['params.qa.few_shot_examples']
    command = f"""
dvc exp run --queue \\
    -S qa.temperature='{temperature}' \\
    -S qa.system_prompt='{system_prompt}' \\
    -S qa.user_prompt_template='{user_prompt_template}' \\
    -S qa.few_shot_examples='{few_shot_examples}' \\
    -S run='{run}'
    """
    return command

In [11]:
with open("../tmp/missing_configs.sh", "w") as f:
    f.write('#!/bin/sh\n')
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n")